In [1]:
from sagemaker.session import production_variant

variant1 = production_variant(
    model_name="CustomerChurn-ab650dc94dbf4da3bef7e06f30649928",
    instance_type="ml.c5.xlarge",
    initial_instance_count=1,
    variant_name="Variant1",
    initial_weight=1,
)
variant2 = production_variant(
    model_name="CustomerChurn-cf3babf7c0d14e6789a5ff35d05102fe",
    instance_type="ml.c5.xlarge",
    initial_instance_count=1,
    variant_name="Variant2",
    initial_weight=1,
)

(variant1, variant2)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


({'ModelName': 'CustomerChurn-ab650dc94dbf4da3bef7e06f30649928',
  'VariantName': 'Variant1',
  'InitialVariantWeight': 1,
  'InitialInstanceCount': 1,
  'InstanceType': 'ml.c5.xlarge'},
 {'ModelName': 'CustomerChurn-cf3babf7c0d14e6789a5ff35d05102fe',
  'VariantName': 'Variant2',
  'InitialVariantWeight': 1,
  'InitialInstanceCount': 1,
  'InstanceType': 'ml.c5.xlarge'})

In [2]:
from sagemaker.session import Session
import boto3
import sagemaker

In [3]:
endpoint_name = f"xgb-churn-pred-abtest"
print(f"EndpointName={endpoint_name}")


region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()
boto_session = boto3.Session(region_name=region)

sagemaker_session.endpoint_from_production_variants(
    name=endpoint_name, production_variants=[variant1, variant2]
)

EndpointName=xgb-churn-pred-abtest
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-west-2:846634201516:endpoint-config/xgb-churn-pred-abtest".

In [4]:
endpoint_name="xgb-churn-pred-abtest"
sagemaker_runtime = boto3.client(
    "sagemaker-runtime", region_name='us-west-2')

response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    TargetVariant='Variant2',
    ContentType='text/csv',
    Body = "1,0,123.8,113,236.2,77,73.2,81,3.7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0"
)
print(response['Body'].read().decode('utf-8'))

0.003065170720219612


In [5]:
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    TargetVariant='Variant1',
    ContentType='text/csv',
    Body = "1,0,123.8,113,236.2,77,73.2,81,3.7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0"
)
print(response['Body'].read().decode('utf-8'))

5.2771760238101706e-05
